In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.cross_validation import train_test_split

## Set up the minibatcher for batch learning

In [2]:
def minibatcher(data_x, data_y, batch_size, num_repeats):
    assert(data_x.shape[0])
    data_size = data_x.shape[0]
    for _ in range(num_repeats):
        start = 0
        while start < data_size:
            yield data_x[start:start + batch_size], data_y[start:start + batch_size]
            start += batch_size
    

## Indicate all of the training constants

In [3]:
classes = ["walking", "sitting", "table", "stairs", "car"]

n_hidden = 100 # Size of the LSTM hidden layer
batch_size = 8 # Number of data points in a batch
learning_rate = 0.01 # Learning rate of the optimizer
dropout_keep_prob = .8


## Import the data, and split it into training and testing sets

In [49]:
clean_data = np.load("gmail/clean_data.npy")
data_labels = np.load("gmail/labels.npy")

# Update the dataset to only be the labeled data (the ones that aren't 0)
labeled = data_labels != 0
input_motion_data = clean_data[labeled]
output_motion_data = data_labels[labeled] - 1 # Need to decrement by 1 since we removed all the 0s
n_samples = input_motion_data.shape[0]
n_steps = input_motion_data.shape[1]
n_input = input_motion_data.shape[2]
n_classes = np.max(output_motion_data) + 1

In [5]:
output_classes = np.zeros((n_samples, n_classes))
for i in range(n_samples):
    output_classes[i, output_motion_data[i]] = 1

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(input_motion_data, output_classes, test_size=.2)

#### Count of each class

In [71]:
zip(classes, np.bincount(np.argmax(Y_train, axis=1)))

[('walking', 54), ('sitting', 48), ('table', 68), ('stairs', 20), ('car', 78)]

## Define the placeholders and variables to be optimized

In [8]:
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_hidden, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Define the model

In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    hidden_layer = tf.nn.relu(tf.matmul(outputs[-1], weights['hidden']) + biases['hidden'])
    hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
    return tf.matmul(hidden_layer, weights['out']) + biases['out']
    
    
    # Linear activation, using rnn inner loop last output
    # return tf.matmul(outputs[-1], weights['out']) + biases['out']


## Define the various graphs: notably cost, optimizer, and accuracy

In [10]:
pred = RNN(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Run the optimization

In [11]:
sess = tf.Session()

In [12]:
sess.run(tf.initialize_all_variables())

for (rep, (batch_x, batch_y)) in enumerate(minibatcher(X_train,Y_train,10, 20)):
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout_keep_prob})
    if rep % 5 == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        test_acc = sess.run(accuracy, feed_dict={x: X_test, y: Y_test, keep_prob: 1.0})
        # Calculate batch loss
        loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout_keep_prob})
        print "Batch " + str(rep) + ", Minibatch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc) + ", Test Accuracy= " + "{:.5f}".format(test_acc)
final_test_acc = sess.run(accuracy, feed_dict={x: X_test, y: Y_test, keep_prob: 1.0})
print "Final Test accuracy = " + "{:.5f}".format(final_test_acc)

Batch 0, Minibatch Loss= 17.240484, Training Accuracy= 0.60000, Test Accuracy= 0.20896
Batch 5, Minibatch Loss= 5.334779, Training Accuracy= 0.80000, Test Accuracy= 0.62687
Batch 10, Minibatch Loss= 6.588993, Training Accuracy= 0.90000, Test Accuracy= 0.64179
Batch 15, Minibatch Loss= 1.040450, Training Accuracy= 0.80000, Test Accuracy= 0.65672
Batch 20, Minibatch Loss= 0.000001, Training Accuracy= 0.90000, Test Accuracy= 0.76119
Batch 25, Minibatch Loss= 3.497866, Training Accuracy= 0.70000, Test Accuracy= 0.77612
Batch 30, Minibatch Loss= 9.008074, Training Accuracy= 0.60000, Test Accuracy= 0.73134
Batch 35, Minibatch Loss= 7.065759, Training Accuracy= 0.50000, Test Accuracy= 0.68657
Batch 40, Minibatch Loss= 1.483553, Training Accuracy= 0.90000, Test Accuracy= 0.67164
Batch 45, Minibatch Loss= 4.219509, Training Accuracy= 0.70000, Test Accuracy= 0.73134
Batch 50, Minibatch Loss= 1.327806, Training Accuracy= 0.90000, Test Accuracy= 0.73134
Batch 55, Minibatch Loss= 0.903301, Training

## Count which classes were confused 

In [72]:
# Get indices of incorrect predictions in the test set
test_predicted = np.argmax(sess.run(pred, feed_dict={x: X_test, keep_prob: 1.0}), axis=1)
test_actual = np.argmax(Y_test, axis=1)
wrong_predictions = test_predicted != test_actual

mistakes = zip(test_predicted[wrong_prediction], test_actual[wrong_predictions])
# Sort the predicted/expected so that mistaking class 1 for class 3 is the same 
# as mistaking class 3 for class 1, for example
mistakes = sorted(map(lambda x: sorted(x), mistakes))
# Convert classs number to class names
mistakes = map(lambda p: (classes[p[0]], classes[p[1]]), mistakes)

from collections import Counter
for m, n in Counter(mistakes).most_common():
    print "%s: %d" % (str(m), n)


('walking', 'car'): 9
('sitting', 'car'): 6
('walking', 'stairs'): 3
('walking', 'sitting'): 1
('stairs', 'car'): 1


## Classify the Unlabelled data

In [73]:
unlabeled = data_labels == 0
input_motion_data = clean_data[unlabeled]

Y = np.argmax(sess.run(pred, feed_dict={x: input_motion_data, keep_prob: 1.0}), axis=1)
class_count = zip(classes, np.bincount(Y))

for c in class_count:
    print "%s: %d" % (c[0], c[1])


walking: 14
sitting: 97
table: 11
stairs: 39
car: 52
